In [1]:
#import findspark
import findspark
findspark.init()

In [2]:

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import string
import re
import pyspark
import matplotlib.pyplot as plt
from pyspark.sql.functions import mean, stddev, col, abs, split, explode
from pyspark.sql import functions as F


from pyspark.ml.feature import IDF,Tokenizer, CountVectorizer, CountVectorizer,StringIndexer,StopWordsRemover
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import * 



In [3]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 01:01:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/28 01:01:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/28 01:01:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/28 01:01:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/28 01:01:05 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/28 01:01:05 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/11/28 01:01:05 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [4]:
## creating sparksession and giving an app name
spark = SparkSession(sc).builder.appName('sparkdf').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



22/11/28 01:01:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
#reading a sample file
yelp_reviews=spark.read.format("csv").option("quote", "\"").option("escape", "\"").option('multiLine', True).option("encoding", "ISO-8859-1").option("header", "true").load("yelp_review.csv")
df_business = spark.read.format("csv").option("quote", "\"").option("escape", "\"").option('multiLine', True).option("encoding", "ISO-8859-1").option("header", "true").load("yelp_business.csv")
df_business=df_business.na.drop(how='any')
df_business.select('business_id','categories').withColumn("categories", explode(split("categories", ";"))).show(10)



+--------------------+--------------------+
|         business_id|          categories|
+--------------------+--------------------+
|kCoE3jvEtg6UVz5SO...|Real Estate Services|
|kCoE3jvEtg6UVz5SO...|         Real Estate|
|kCoE3jvEtg6UVz5SO...|       Home Services|
|kCoE3jvEtg6UVz5SO...| Property Management|
|xcgFnd-MwkZeO5G2H...|            Bakeries|
|xcgFnd-MwkZeO5G2H...|              Bagels|
|xcgFnd-MwkZeO5G2H...|                Food|
|fNMVV_ZX7CJSDWQGd...|         Restaurants|
|fNMVV_ZX7CJSDWQGd...|American (Traditi...|
|l09JfMeQ6ynYs5MCJ...|             Italian|
+--------------------+--------------------+
only showing top 10 rows



In [6]:
##  Unique Business Categories

print("****************Unique Categories********************************\n")
unique_categories = df_business.select('business_id','categories').withColumn("categories", explode(split("categories", ", "))).select('categories').distinct().count()
print(f'Unique: {unique_categories}')
print("\n")

print("****************Unique Categories********************************\n")



****************Unique Categories********************************



Unique: 33316


****************Unique Categories********************************



In [7]:
df_business.select("categories").show()

+--------------------+
|          categories|
+--------------------+
|Real Estate Servi...|
|Bakeries;Bagels;Food|
|Restaurants;Ameri...|
|Italian;French;Re...|
|   Food;Coffee & Tea|
|Bars;Sports Bars;...|
|Tiki Bars;Nightli...|
|Shopping;Jewelry;...|
|Coffee & Tea;Food...|
|Arts & Entertainm...|
|Hair Salons;Blow ...|
|Home Services;Lig...|
|Tanning;Day Spas;...|
| Italian;Restaurants|
|Hotels & Travel;C...|
|Nightlife;Bars;Ba...|
|Fast Food;Restaur...|
|Arcades;Arts & En...|
| Restaurants;Mexican|
|Active Life;Fitne...|
+--------------------+
only showing top 20 rows

